In [1]:
import numpy as np
import pandas as pd

# 2.1 Retrieve Reddit World News

In [2]:
news_data = pd.read_csv('./data/RedditNews.csv', encoding='utf-8')
news_data.head()
news_data.shape

(73608, 2)

### Strip binary data and newline tags from news data and eliminate any data not between dates (1/4/2010)

In [3]:
news_data.rename(columns={'Date': 'date'}, inplace=True)
news_data = news_data[news_data['date'] > '2010-01-04']
news_data = news_data.replace({"b'": "", 'b"': '', '\n': '', '\t': '', '"': ''}, regex=True)

## 2.1.1 Combine news articles by date

In [4]:
news_data = news_data.groupby(['date'])['News'].apply(lambda x: ', '.join(x)).reset_index()
news_data.head()

,date,News
0,2010-01-05,These images depict the untouched stomach cont...
1,2010-01-06,Three Americans go to Uganda and teach thousan...
2,2010-01-07,23-year-old British woman on holiday in Dubai ...
3,2010-01-08,Top Imams affiliated with the Islamic Supreme ...
4,2010-01-09,"Chevron could face damages of $27.3bn, the big..."


# 2.2 Retrieve and Combine Stocks and Securities

In [6]:
pdata = pd.read_csv('./data/nyse/prices-split-adjusted.csv', encoding='utf-8')
pdata.head()

,date,symbol,open,close,low,high,volume
0,2016-01-05,WLTW,123.430000,125.839996,122.309998,126.250000,2163600.0
1,2016-01-06,WLTW,125.239998,119.980003,119.940002,125.540001,2386400.0
2,2016-01-07,WLTW,116.379997,114.949997,114.930000,119.739998,2489500.0
3,2016-01-08,WLTW,115.480003,116.620003,113.500000,117.440002,2006300.0
4,2016-01-11,WLTW,117.010002,114.970001,114.089996,117.330002,1408600.0


In [7]:
secdata = pd.read_csv('./data/nyse/securities.csv', encoding='utf-8')
# rename column to allow join
secdata.rename(columns={'Ticker symbol': 'symbol'}, inplace=True)
secdata.head()

,symbol,Security,SEC filings,GICS Sector,GICS Sub Industry,Address of Headquarters,Date first added,CIK
0,MMM,3M Company,reports,Industrials,Industrial Conglomerates,"St. Paul, Minnesota",NaN,66740
1,ABT,Abbott Laboratories,reports,Health Care,Health Care Equipment,"North Chicago, Illinois",1964-03-31,1800
2,ABBV,AbbVie,reports,Health Care,Pharmaceuticals,"North Chicago, Illinois",2012-12-31,1551152
3,ACN,Accenture plc,reports,Information Technology,IT Consulting & Other Services,"Dublin, Ireland",2011-07-06,1467373
4,ATVI,Activision Blizzard,reports,Information Technology,Home Entertainment Software,"Santa Monica, California",2015-08-31,718877


In [8]:
stock_data = pd.merge(pdata, secdata[['symbol','Security','GICS Sector', 'GICS Sub Industry']], on='symbol')
stock_data.head()

,date,symbol,open,close,low,high,volume,Security,GICS Sector,GICS Sub Industry
0,2016-01-05,WLTW,123.430000,125.839996,122.309998,126.250000,2163600.0,Willis Towers Watson,Financials,Insurance Brokers
1,2016-01-06,WLTW,125.239998,119.980003,119.940002,125.540001,2386400.0,Willis Towers Watson,Financials,Insurance Brokers
2,2016-01-07,WLTW,116.379997,114.949997,114.930000,119.739998,2489500.0,Willis Towers Watson,Financials,Insurance Brokers
3,2016-01-08,WLTW,115.480003,116.620003,113.500000,117.440002,2006300.0,Willis Towers Watson,Financials,Insurance Brokers
4,2016-01-11,WLTW,117.010002,114.970001,114.089996,117.330002,1408600.0,Willis Towers Watson,Financials,Insurance Brokers


### Evaluate data and find out how many stocks in energy/oil sector

In [9]:
stock_data[(stock_data['date'] == '2016-01-05') & (stock_data['GICS Sector'] == 'Energy')].describe()

,open,close,low,high,volume
count,36.000000,36.000000,36.000000,36.000000,3.600000e+01
mean,47.807778,47.860000,46.929166,48.365556,7.044572e+06
std,29.533748,29.666613,29.209861,29.905739,6.586727e+06
min,4.890000,5.010000,4.840000,5.090000,1.448800e+06
25%,25.602500,25.595000,24.917499,25.817501,2.777550e+06
50%,45.084999,44.724998,43.850000,45.774999,5.129350e+06
75%,69.519997,69.657499,68.209997,70.014997,8.344725e+06
max,124.349998,125.169998,122.839996,126.089996,2.911390e+07


## 2.3 Group by data and securities

In [10]:
grp_data = stock_data.groupby(['date','GICS Sector']).agg({
    'Security': "count",
    "open": sum,
    "close": sum
})
grp_data.head()

Security         open        close
date       GICS Sector                                               
2010-01-04 Consumer Discretionary        77  2672.473763  2671.436019
           Consumer Staples              34  1167.468720  1171.816422
           Energy                        33  1525.897759  1551.516528
           Financials                    58  2110.190096  2128.708649
           Health Care                   54  2391.403717  2403.706137

In [11]:
grp_data.xs(key='Energy', level=1).head()

,Security,open,close
date,,,
2010-01-04,33,1525.897759,1551.516528
2010-01-05,33,1551.832163,1566.911977
2010-01-06,33,1565.173122,1584.721043
2010-01-07,33,1577.543497,1579.518125
2010-01-08,33,1572.397667,1594.940442


### Similar the DJIA index methodology to get the average of the stocks by sector

In [12]:
# data was previously summed in previous row
grp_data['open']=grp_data['open']/grp_data['Security']
grp_data['close']=grp_data['close']/grp_data['Security']

In [13]:
grp_data['Daily Return']=(grp_data['close']-grp_data['open'])/grp_data['open']*100

In [171]:
grp_data.xs(key='Energy', level=1).head()

,Security,open,close,Daily Return
date,,,,
2010-01-04,33,46.239326,47.015652,1.678931
2010-01-05,33,47.025217,47.482181,0.971743
2010-01-06,33,47.429489,48.021850,1.248930
2010-01-07,33,47.804348,47.864186,0.125171
2010-01-08,33,47.648414,48.331529,1.433656


## 2.4 Calculate Daily Returns and Standard Deviations

### Calculate the means and standard deviations by sector

In [172]:
sigma = grp_data.groupby('GICS Sector').agg({'Daily Return':['mean', 'std']})
sigma

Daily Return          
                                    mean       std
GICS Sector                                       
Consumer Discretionary          0.036876  0.929677
Consumer Staples                0.051984  0.610403
Energy                          0.013425  1.295804
Financials                      0.032340  1.005465
Health Care                     0.043555  0.877931
Industrials                     0.046591  0.883032
Information Technology          0.023120  0.913375
Materials                       0.029144  0.947435
Real Estate                     0.043639  0.997881
Telecommunications Services    -0.005438  0.853520
Utilities                       0.036095  0.794455

### Generate the 1 and 2 sigma lower and upper limits

In [173]:
sigma['1sigmalow']=sigma.iloc[:,0]-sigma.iloc[:,1]
sigma['1sigmahi']=sigma.iloc[:,0]+sigma.iloc[:,1]
sigma['2sigmalow']=sigma.iloc[:,0]-(2*sigma.iloc[:,1])
sigma['2sigmahi']=sigma.iloc[:,0]+(2*sigma.iloc[:,1])
sigma

Daily Return           1sigmalow  1sigmahi  \
                                    mean       std                       
GICS Sector                                                              
Consumer Discretionary          0.036876  0.929677 -0.892801  0.966552   
Consumer Staples                0.051984  0.610403 -0.558419  0.662387   
Energy                          0.013425  1.295804 -1.282379  1.309229   
Financials                      0.032340  1.005465 -0.973125  1.037805   
Health Care                     0.043555  0.877931 -0.834376  0.921485   
Industrials                     0.046591  0.883032 -0.836441  0.929623   
Information Technology          0.023120  0.913375 -0.890255  0.936495   
Materials                       0.029144  0.947435 -0.918291  0.976578   
Real Estate                     0.043639  0.997881 -0.954241  1.041520   
Telecommunications Services    -0.005438  0.853520 -0.858958  0.848081   
Utilities                       0.036095  0.794455 -0.758360  0.830551   

                            2sigmalow  2sigmahi  
                                                 
GICS Sector                                      
Consumer Discretionary      -1.822478  1.896229  
Consumer Staples            -1.168822  1.272791  
Energy                      -2.578183  2.605033  
Financials                  -1.978589  2.043269  
Health Care                 -1.712307  1.799416  
Industrials                 -1.719473  1.812655  
Information Technology      -1.803631  1.849871  
Materials                   -1.865725  1.924013  
Real Estate                 -1.952122  2.039400  
Telecommunications Services -1.712478  1.701601  
Utilities                   -1.552815  1.625006

## 2.5 Generate Labels based on whether sigma threshold passed

### Setup labels for 1 and 2 sigma threshold of prices.

In [279]:
def onesigma(row):
    if row['Daily Return'] < sigma.xs(row['GICS Sector'])['1sigmalow'][0]:
        row['Label1Sig'] = -1
    elif row['Daily Return'] > sigma.xs(row['GICS Sector'])['1sigmahi'][0]:
        row['Label1Sig'] = 1
    else:
        row['Label1Sig'] = 0
    if row['Daily Return'] < sigma.xs(row['GICS Sector'])['2sigmalow'][0]:
        row['Label2Sig'] = -1
    elif row['Daily Return'] > sigma.xs(row['GICS Sector'])['2sigmahi'][0]:
        row['Label2Sig'] = 1
    else:
        row['Label2Sig'] = 0
    return row


In [260]:
grp_data['Label1Sig'] = 0
grp_data['Label2Sig'] = 0

In [281]:
new_df = grp_data.apply(onesigma, axis=1)

In [286]:
new_df[new_df['GICS Sector'] == 'Energy'].sort_values(by=['Daily Return'], ascending=False).head()

,level_0,index,date,GICS Sector,Security,open,close,Daily Return,Label1Sig,Label2Sig
15919,15919,15919,2015-10-02,Energy,36,49.122500,51.973611,5.804083,1,1
4864,4864,4864,2011-10-04,Energy,33,42.634050,45.061645,5.694030,1,1
13257,13257,13257,2014-10-16,Energy,36,62.619371,65.776134,5.041193,1,1
1058,1058,1058,2010-05-21,Energy,33,40.388387,42.200942,4.487812,1,1
13730,13730,13730,2014-12-17,Energy,36,57.999861,60.550138,4.397041,1,1


In [302]:
combined_df = pd.merge(news_data, new_df[['date','GICS Sector', 'Label1Sig', 'Label2Sig']], on='date')

In [303]:
combined_df.to_csv (r'.\cache\data.csv', index = None, header=True) 

In [304]:
combined_df.shape

(17985, 5)